# Setting up

In [1]:
from __future__ import print_function

# Import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import psycopg2
import os

# below is used to print out pretty pandas dataframes
from IPython.display import display, HTML

# specify user/password/where the database is
sqluser = 'sharing'
sqlpass = 'mimicdatabase'
dbname = 'mimicdatabase'
schema_name = 'mimiciii'
host = 'localhost'

# connect to the database
con = psycopg2.connect(dbname=dbname, user=sqluser, password=sqlpass, host=host)
# query_schema = 'set search_path to ' + schema_name + ';'
cur = con.cursor()
cur.execute('SET search_path to {}'.format(schema_name))

In [2]:
def execute_query_safely(sql, con):
    cur = con.cursor()
    
    # try to execute the query
    try:
        cur.execute(sql)
    except:
        # if an exception, rollback, rethrow the exception - finally closes the connection
        cur.execute('rollback;')
        raise
    finally:
        cur.close()
    
    return

concepts_path = '/home/sharing/concepts/'
query_schema = 'set search_path to ' + schema_name + ';'
queries = ['postgres-functions.sql'#,
           #'firstday/urine_output_first_day.sql',
           #'firstday/vitals_first_day.sql',
           #'firstday/gcs_first_day.sql',
           #'firstday/labs_first_day.sql',
           #'firstday/blood_gas_first_day.sql',
           #'firstday/blood_gas_first_day_arterial.sql',
           #'durations/ventilation_classification.sql',  
           #'durations/ventilation_durations.sql'          
          ]

for query_file in queries:
    # load in the text of the query
    f = os.path.join(concepts_path, query_file)
    with open(f) as fp:
        query = ''.join(fp.readlines())

    # Execute the query
    print('Generating materialized view using {} ...'.format(f),end=' ')
    execute_query_safely(query_schema + query, con)
    print('done.')

Generating materialized view using /home/sharing/concepts/postgres-functions.sql ... done.


# Get subject_id, gender, age 

In [3]:
# Get subject_id, gender, age, service type and length of stay from the icustays table


query = \
"""
SELECT ie.subject_id, ie.icustay_id,
    ie.intime, pat.GENDER,
    ROUND((cast(ie.intime as date) - cast(pat.dob as date))/365.242, 2) AS age,
    EXTRACT(DAY FROM ie.outtime - ie.intime) AS length_of_stay,
    ie.first_careunit AS service_type
FROM icustays ie
INNER JOIN patients pat
ON ie.subject_id = pat.subject_id;
"""

data = pd.read_sql_query(query,con)
data

,subject_id,icustay_id,intime,gender,age,length_of_stay,service_type
0,2,243653,2138-07-17 21:20:00,M,0.00,0.0,NICU
1,3,211552,2101-10-20 19:10:00,M,76.52,6.0,MICU
2,4,294638,2191-03-16 00:29:00,F,47.84,1.0,MICU
3,5,214757,2103-02-02 06:04:00,M,0.00,0.0,NICU
4,6,228232,2175-05-30 21:30:00,F,65.94,3.0,SICU
...,...,...,...,...,...,...,...
61527,99985,279638,2181-01-29 05:33:00,M,53.81,11.0,MICU
61528,99991,226241,2184-12-28 17:30:00,M,47.73,3.0,TSICU
61529,99992,242052,2144-07-25 18:04:00,F,65.77,1.0,MICU
61530,99995,229633,2147-02-08 13:53:00,F,88.70,2.0,CSRU


In [4]:
demo = data[["icustay_id", "gender", "age", "length_of_stay", "service_type"]]
demo

,icustay_id,gender,age,length_of_stay,service_type
0,243653,M,0.00,0.0,NICU
1,211552,M,76.52,6.0,MICU
2,294638,F,47.84,1.0,MICU
3,214757,M,0.00,0.0,NICU
4,228232,F,65.94,3.0,SICU
...,...,...,...,...,...
61527,279638,M,53.81,11.0,MICU
61528,226241,M,47.73,3.0,TSICU
61529,242052,F,65.77,1.0,MICU
61530,229633,F,88.70,2.0,CSRU


# Get weight

In [5]:
# Get weight_adults from the chartevents table

query = \
"""
    SELECT
        c.icustay_id
      , -- c.charttime
      case when c.charttime <= (ie.outtime) then c.charttime end as time
      , case when c.itemid in (762,226512) then 'admit'
          else 'daily' end as weight_type
      -- TODO: eliminate obvious outliers if there is a reasonable weight
      , c.valuenum as weight
    FROM chartevents c
    INNER JOIN icustays ie
    ON c.icustay_id = ie.icustay_id
    WHERE c.valuenum IS NOT NULL
      AND c.itemid in
      (
          762,226512 -- Admit Wt
      )
      AND c.icustay_id IS NOT NULL
      AND c.valuenum > 0
      -- exclude rows marked as error
      AND COALESCE(c.error, 0) = 0
"""

data_adults = pd.read_sql_query(query,con)

In [6]:
df_adults = data_adults.sort_values(['icustay_id','time'], ascending = (True, False)).drop_duplicates(subset='icustay_id', keep="first")
df_adults

,icustay_id,time,weight_type,weight
38780,200001,2181-11-25 19:17:00,admit,61.000000
26170,200003,2199-08-03 17:51:00,admit,77.000000
19675,200007,2109-02-17 11:31:00,admit,126.000000
28102,200009,2189-12-01 11:01:00,admit,78.199997
30782,200010,2132-08-05 00:45:00,admit,49.300000
...,...,...,...,...
12537,299992,2105-11-22 00:42:00,admit,90.000000
10285,299994,2114-12-12 14:28:00,admit,72.599998
27202,299995,2116-03-05 01:30:00,admit,70.400002
42849,299998,2181-07-05 18:55:00,admit,90.000000


In [7]:
# Get weight_neonates from the chartevents table

query = \
"""
    SELECT c.icustay_id, --c.charttime
    case when c.charttime <= (ie.outtime) then c.charttime end as time
    , MAX(CASE WHEN c.itemid = 3580 THEN c.valuenum END) as wt_kg
    , MAX(CASE WHEN c.itemid = 3581 THEN c.valuenum END) as wt_lb
    , MAX(CASE WHEN c.itemid = 3582 THEN c.valuenum END) as wt_oz
    FROM chartevents c 
    INNER JOIN icustays ie
    ON c.icustay_id = ie.icustay_id
    WHERE c.itemid in (3580, 3581, 3582)
    AND c.icustay_id IS NOT NULL
    AND COALESCE(c.error, 0) = 0
    -- wt_oz/wt_lb/wt_kg are only 0 erroneously, so drop these rows
    AND c.valuenum > 0
  -- a separate query was run to manually verify only 1 value exists per
  -- icustay_id/charttime/itemid grouping
  -- therefore, we can use max() across itemid to collapse these values to 1 row per group
    GROUP BY c.icustay_id, c.charttime, ie.outtime
"""

data_neonates = pd.read_sql_query(query,con)

In [8]:
df_neonates = data_neonates.sort_values(['icustay_id','time'], ascending = (True, False)).drop_duplicates(subset='icustay_id', keep="first")
df_neonates

,icustay_id,time,wt_kg,wt_lb,wt_oz
108,200017,2138-05-14 09:00:00,2.745,6.00063,0.826032
109,200020,2174-05-13 23:30:00,4.430,9.00038,12.262000
220,200037,2141-08-25 12:00:00,2.145,4.00087,11.661900
699,200046,2154-07-26 14:30:00,2.780,6.00079,2.060610
708,200058,2118-07-21 19:00:00,2.165,4.00096,12.367300
...,...,...,...,...,...
431712,299922,2142-08-28 07:00:00,1.405,3.00046,1.559410
431720,299940,2132-09-27 12:00:00,2.190,4.00007,13.249200
431964,299952,2170-03-06 13:00:00,2.750,6.00065,1.002400
432185,299963,2133-03-10 13:00:00,3.305,7.00020,4.579250


In [9]:
df_adults_portion = df_adults[['icustay_id','weight']]
df_neonates.rename({'wt_kg': 'weight'}, axis=1, inplace=True)
df_neonates_portion = df_neonates[['icustay_id','weight']]
df_weight = pd.concat([df_adults_portion, df_neonates_portion], ignore_index=True)
df_weight

,icustay_id,weight
0,200001,61.000000
1,200003,77.000000
2,200007,126.000000
3,200009,78.199997
4,200010,49.300000
...,...,...
47793,299922,1.405000
47794,299940,2.190000
47795,299952,2.750000
47796,299963,3.305000


# Get height

In [10]:
# Get heights from chartevents table

query = \
"""
SELECT ie.icustay_id, c. value, c.valueuom,
case when c.charttime <= (ie.outtime) then c.charttime end as time
FROM icustays ie
INNER JOIN chartevents c
ON ie.icustay_id = c.icustay_id
WHERE c.itemid IN (920, 1394, 3486, 226707)
"""

data_height = pd.read_sql_query(query,con)

In [11]:
df = data_height.sort_values(['icustay_id','time'], ascending = (True, False)).dropna().drop_duplicates(subset='icustay_id', keep="first")
df

,icustay_id,value,valueuom,time
46842,200001,67,Inch,2181-11-28 15:15:00
26871,200007,70,inches,2109-02-17 11:31:00
38858,200009,63,inches,2189-12-01 11:01:00
12627,200014,66,inches,2105-02-16 23:34:00
25004,200025,76,inches,2113-08-25 09:13:00
...,...,...,...,...
21844,299984,63,inches,2185-09-20 05:29:00
15474,299988,68,inches,2190-02-12 23:28:00
17134,299992,73,inches,2105-11-22 00:42:00
14037,299994,62,inches,2114-12-12 14:28:00


In [12]:
df["valueuom"].unique()

array(['Inch', 'inches', 'Inches', 'In'], dtype=object)

In [13]:
df.rename({'value': 'height_inches'}, axis=1, inplace=True)
df_height = df[['icustay_id','height_inches']]
df_height

,icustay_id,height_inches
46842,200001,67
26871,200007,70
38858,200009,63
12627,200014,66
25004,200025,76
...,...,...
21844,299984,63
15474,299988,68
17134,299992,73
14037,299994,62


# Get mortality indicator

In [14]:
# Get 28 days mortality indicator from icustay and patients table

query = \
"""
SELECT ie.icustay_id, 
case when pat.dod <= (ie.outtime + interval '28' day) then 1 else 0 end as day_28_flag
FROM icustays ie
INNER JOIN patients pat
ON ie.subject_id = pat.subject_id
"""

months = pd.read_sql_query(query,con)
months

,icustay_id,day_28_flag
0,243653,0
1,211552,0
2,294638,0
3,214757,0
4,228232,0
...,...,...
61527,279638,0
61528,226241,0
61529,242052,0
61530,229633,0


In [15]:
# Get current ICU mortality indicator and ethnicity for CKD-EPI Creatinine Equation from admission and icustay table

query = \
"""
SELECT ie.icustay_id,
case when adm.deathtime <= ie.outtime then 1 else 0 end as icu_exp_flag,
adm.ethnicity
FROM icustays ie
INNER JOIN admissions adm
ON ie.subject_id = adm.subject_id
"""

data = pd.read_sql_query(query,con)

In [16]:
current = data.drop_duplicates(subset='icustay_id')
current

,icustay_id,icu_exp_flag,ethnicity
0,243653,0,ASIAN
1,211552,0,WHITE
2,294638,0,WHITE
3,214757,0,ASIAN
4,228232,0,WHITE
...,...,...,...
116421,279638,0,WHITE
116422,226241,0,WHITE
116423,242052,0,WHITE
116424,229633,0,WHITE


# Get vitals

In [17]:
# Get vitals from chartevents and icustays table

query = \
"""
select ie.icustay_id,
case -- MAP, Temperature, HR, SysBP, DiasBP, SpO2,
when itemid in (456,52,6702,443,220052,220181,225312) then 'MAP'
when itemid in (223762,676,223761,678) then 'Temperature'
when itemid in (51,442,455,6701,220179,220050) and valuenum > 0 and valuenum < 400 then 'SysBP'
when itemid in (8368,8440,8441,8555,220180,220051) and valuenum > 0 and valuenum < 300 then 'DiasBP'
when itemid in (211,220045) then 'HeartRate'
when itemid in (646,220277) then 'SpO2'
else null end as label, 
case 
when itemid in (223761,678) and ((valuenum-32)/1.8)<10 then null
when itemid in (223762,676) and valuenum < 10 then null
-- convert F to C
when itemid in (223761,678) then (valuenum-32)/1.8
-- sanity checks on data - one outliter with spo2 < 25
when itemid in (646,220277) and valuenum <= 25 then null
else valuenum end as valuenum
from icustays ie
inner join chartevents c
on ie.icustay_id = c.icustay_id
    and itemid in
    (
        456,52,6702,443,220052,220181,225312 -- map
      , 223762,676,223761,678 -- temp
      , 51,442,455,6701,220179,220050 -- SysBP
      , 8368,8440,8441,8555,220180,220051 -- DiasBP
      , 211,220045 -- hr
      , 646,220277 -- spo2
    )
    and valuenum is not null
    and coalesce(error,0) != 1
"""

data = pd.read_sql_query(query,con)
data

,icustay_id,label,valuenum
0,294638,SpO2,100.000000
1,294638,Temperature,36.666667
2,263738,SpO2,98.000000
3,263738,Temperature,37.200001
4,263738,SpO2,100.000000
...,...,...,...
34143523,290902,DiasBP,44.000000
34143524,290902,DiasBP,38.000000
34143525,290902,DiasBP,38.000000
34143526,290902,DiasBP,34.000000


In [18]:
df = data.groupby(['icustay_id','label']).median().unstack('label')
df.columns = df.columns.get_level_values(-1)
vitals = df.reset_index()
vitals

label,icustay_id,DiasBP,HeartRate,MAP,SpO2,SysBP,Temperature
0,200001,55.0,84.0,69.000000,99.0,107.5,36.555556
1,200003,62.0,78.0,78.000000,97.0,108.0,37.555555
2,200006,53.5,75.0,69.833351,99.0,107.5,37.111113
3,200007,60.5,96.0,79.833298,96.0,119.0,37.222222
4,200009,61.0,98.0,73.000000,100.0,104.0,37.700001
...,...,...,...,...,...,...,...
60184,299993,71.0,114.0,96.000000,95.0,153.0,37.333332
60185,299994,48.0,70.0,72.000000,98.0,121.0,37.200001
60186,299995,64.0,89.0,80.500000,98.0,122.0,36.999999
60187,299998,55.0,80.5,78.000000,98.0,137.5,36.944444


# Get labs

In [19]:
# Get labs from icustays and labevents table

query = \
"""
select ie.icustay_id, l.valuenum, --l.charttime,
case when l.charttime <= (ie.outtime) then l.charttime end as time
, case
when itemid = 51006 then 'BUN'
when itemid = 50806 then 'CHLORIDE'
when itemid = 50902 then 'CHLORIDE'
when itemid = 50912 then 'CREATININE'
when itemid = 50811 then 'HEMOGLOBIN'
when itemid = 51222 then 'HEMOGLOBIN'
when itemid = 51265 then 'PLATELET'
when itemid = 50822 then 'POTASSIUM'
when itemid = 50971 then 'POTASSIUM'
when itemid = 50824 then 'SODIUM'
when itemid = 50983 then 'SODIUM'
when itemid = 50803 then 'TOTALCO2' -- actually is 'BICARBONATE'
when itemid = 50882 then 'TOTALCO2' -- actually is 'BICARBONATE'
when itemid = 50804 then 'TOTALCO2'
when itemid = 51300 then 'WBC'
when itemid = 51301 then 'WBC'
else null end as label
from icustays ie
inner join labevents l
on ie.subject_id = l.subject_id
where l.itemid in
(
51300,51301 -- wbc
,50811,51222 -- hgb
,51265 -- platelet
,50824, 50983 -- sodium
,50822, 50971 -- potassium
,50804 -- Total CO2 or ...
,50803, 50882  -- bicarbonate
,50806,50902 -- chloride
,51006 -- bun
,50912 -- creatinine
)
and valuenum is not null
"""

data = pd.read_sql_query(query,con)
data

,icustay_id,valuenum,time,label
0,211552,22.0,2101-10-12 18:17:00,TOTALCO2
1,211552,23.0,2101-10-13 03:00:00,TOTALCO2
2,211552,109.0,2101-10-13 03:00:00,CHLORIDE
3,211552,1.7,2101-10-13 03:00:00,CREATININE
4,211552,4.3,2101-10-13 03:00:00,POTASSIUM
...,...,...,...,...
18779618,233856,33.0,2109-12-29 22:11:00,TOTALCO2
18779619,233856,29.0,2109-12-29 23:04:00,TOTALCO2
18779620,233856,26.0,2109-12-30 01:40:00,TOTALCO2
18779621,233856,97.0,2109-12-30 01:40:00,CHLORIDE


In [20]:
df = data.sort_values(["icustay_id","label","time"], ascending = (True, True, False)).dropna().groupby(['icustay_id','label']).first().unstack()
df = df[df.columns[0:9]]
df

valuenum                                                           \
label           BUN CHLORIDE CREATININE HEMOGLOBIN PLATELET POTASSIUM SODIUM   
icustay_id                                                                     
200001         58.0     98.0        2.9        8.4    168.0       4.2  135.0   
200003         11.0    111.0        0.7        9.6    116.0       3.8  144.0   
200006         14.0     98.0        0.8        8.7    162.0       3.5  137.0   
200007          8.0    101.0        0.8       12.8    225.0       3.9  135.0   
200009         21.0    112.0        0.5        9.0     75.0       4.2  141.0   
...             ...      ...        ...        ...      ...       ...    ...   
299993         12.0    106.0        0.5        9.6    370.0       3.3  137.0   
299994         35.0    106.0        3.9       10.1    110.0       4.5  138.0   
299995          6.0    104.0        0.8       14.9    234.0       3.8  142.0   
299998         20.0    110.0        1.0        9.2    188.0       4.1  141.0   
299999         13.0    106.0        1.0       11.5    290.0       4.5  139.0   

                           
label      TOTALCO2   WBC  
icustay_id                 
200001         27.0   3.4  
200003         28.0  14.5  
200006         31.0   4.2  
200007         24.0   9.4  
200009         25.0  12.8  
...             ...   ...  
299993         26.0  13.3  
299994         20.0   7.8  
299995         27.0  13.3  
299998         25.0  11.0  
299999         25.0  15.9  

[60102 rows x 9 columns]

In [21]:
df.columns = df.columns.get_level_values(-1)
labs = df.reset_index()
labs

label,icustay_id,BUN,CHLORIDE,CREATININE,HEMOGLOBIN,PLATELET,POTASSIUM,SODIUM,TOTALCO2,WBC
0,200001,58.0,98.0,2.9,8.4,168.0,4.2,135.0,27.0,3.4
1,200003,11.0,111.0,0.7,9.6,116.0,3.8,144.0,28.0,14.5
2,200006,14.0,98.0,0.8,8.7,162.0,3.5,137.0,31.0,4.2
3,200007,8.0,101.0,0.8,12.8,225.0,3.9,135.0,24.0,9.4
4,200009,21.0,112.0,0.5,9.0,75.0,4.2,141.0,25.0,12.8
...,...,...,...,...,...,...,...,...,...,...
60097,299993,12.0,106.0,0.5,9.6,370.0,3.3,137.0,26.0,13.3
60098,299994,35.0,106.0,3.9,10.1,110.0,4.5,138.0,20.0,7.8
60099,299995,6.0,104.0,0.8,14.9,234.0,3.8,142.0,27.0,13.3
60100,299998,20.0,110.0,1.0,9.2,188.0,4.1,141.0,25.0,11.0


# Get Risk Score

In [22]:
# Get SOFA Score coma score from sofa.sql in repository

query = \
"""
with wt AS
(
  SELECT ie.icustay_id
    -- ensure weight is measured in kg
    , avg(CASE
        WHEN itemid IN (762, 763, 3723, 3580, 226512)
          THEN valuenum
        -- convert lbs to kgs
        WHEN itemid IN (3581)
          THEN valuenum * 0.45359237
        WHEN itemid IN (3582)
          THEN valuenum * 0.0283495231
        ELSE null
      END) AS weight

  FROM icustays ie
  left join chartevents c
    on ie.icustay_id = c.icustay_id
  WHERE valuenum IS NOT NULL
  AND itemid IN
  (
    762, 763, 3723, 3580,                     -- Weight Kg
    3581,                                     -- Weight lb
    3582,                                     -- Weight oz
    226512 -- Metavision: Admission Weight (Kg)
  )
  AND valuenum != 0
  and charttime between DATETIME_SUB(ie.intime, INTERVAL '1' DAY) and DATETIME_ADD(ie.intime, INTERVAL '1' DAY)
  -- exclude rows marked as error
  AND (c.error IS NULL OR c.error = 0)
  group by ie.icustay_id
)
-- 5% of patients are missing a weight, but we can impute weight using their echo notes
, echo2 as(
  select ie.icustay_id, avg(weight * 0.45359237) as weight
  FROM icustays ie
  left join echo_data echo
    on ie.hadm_id = echo.hadm_id
    and echo.charttime > DATETIME_SUB(ie.intime, INTERVAL '7' DAY)
    and echo.charttime < DATETIME_ADD(ie.intime, INTERVAL '1' DAY)
  group by ie.icustay_id
)
, vaso_cv as
(
  select ie.icustay_id
    -- case statement determining whether the ITEMID is an instance of vasopressor usage
    , max(case
            when itemid = 30047 then rate / coalesce(wt.weight,ec.weight) -- measured in mcgmin
            when itemid = 30120 then rate -- measured in mcgkgmin ** there are clear errors, perhaps actually mcgmin
            else null
          end) as rate_norepinephrine

    , max(case
            when itemid =  30044 then rate / coalesce(wt.weight,ec.weight) -- measured in mcgmin
            when itemid in (30119,30309) then rate -- measured in mcgkgmin
            else null
          end) as rate_epinephrine

    , max(case when itemid in (30043,30307) then rate end) as rate_dopamine
    , max(case when itemid in (30042,30306) then rate end) as rate_dobutamine

  FROM icustays ie
  inner join inputevents_cv cv
    on ie.icustay_id = cv.icustay_id and cv.charttime between ie.intime and DATETIME_ADD(ie.intime, INTERVAL '1' DAY)
  left join wt
    on ie.icustay_id = wt.icustay_id
  left join echo2 ec
    on ie.icustay_id = ec.icustay_id
  where itemid in (30047,30120,30044,30119,30309,30043,30307,30042,30306)
  and rate is not null
  group by ie.icustay_id
)
, vaso_mv as
(
  select ie.icustay_id
    -- case statement determining whether the ITEMID is an instance of vasopressor usage
    , max(case when itemid = 221906 then rate end) as rate_norepinephrine
    , max(case when itemid = 221289 then rate end) as rate_epinephrine
    , max(case when itemid = 221662 then rate end) as rate_dopamine
    , max(case when itemid = 221653 then rate end) as rate_dobutamine
  FROM icustays ie
  inner join inputevents_mv mv
    on ie.icustay_id = mv.icustay_id and mv.starttime between ie.intime and DATETIME_ADD(ie.intime, INTERVAL '1' DAY)
  where itemid in (221906,221289,221662,221653)
  -- 'Rewritten' orders are not delivered to the patient
  and statusdescription != 'Rewritten'
  group by ie.icustay_id
)
, pafi1 as
(
  -- join blood gas to ventilation durations to determine if patient was vent
  select bg.icustay_id, bg.charttime
  , pao2fio2
  , case when vd.icustay_id is not null then 1 else 0 end as isvent
  from blood_gas_first_day_arterial bg
  left join ventilation_durations vd
    on bg.icustay_id = vd.icustay_id
    and bg.charttime >= vd.starttime
    and bg.charttime <= vd.endtime
  order by bg.icustay_id, bg.charttime
)
, pafi2 as
(
  -- because pafi has an interaction between vent/PaO2:FiO2, we need two columns for the score
  -- it can happen that the lowest unventilated PaO2/FiO2 is 68, but the lowest ventilated PaO2/FiO2 is 120
  -- in this case, the SOFA score is 3, *not* 4.
  select icustay_id
  , min(case when isvent = 0 then pao2fio2 else null end) as pao2fio2_novent_min
  , min(case when isvent = 1 then pao2fio2 else null end) as pao2fio2_vent_min
  from pafi1
  group by icustay_id
)
-- Aggregate the components for the score
, scorecomp as
(
select ie.icustay_id
  , v.meanbp_min
  , coalesce(cv.rate_norepinephrine, mv.rate_norepinephrine) as rate_norepinephrine
  , coalesce(cv.rate_epinephrine, mv.rate_epinephrine) as rate_epinephrine
  , coalesce(cv.rate_dopamine, mv.rate_dopamine) as rate_dopamine
  , coalesce(cv.rate_dobutamine, mv.rate_dobutamine) as rate_dobutamine

  , l.creatinine_max
  , l.bilirubin_max
  , l.platelet_min

  , pf.pao2fio2_novent_min
  , pf.pao2fio2_vent_min

  , uo.urineoutput

  , gcs.mingcs
FROM icustays ie
left join vaso_cv cv
  on ie.icustay_id = cv.icustay_id
left join vaso_mv mv
  on ie.icustay_id = mv.icustay_id
left join pafi2 pf
 on ie.icustay_id = pf.icustay_id
left join vitals_first_day v
  on ie.icustay_id = v.icustay_id
left join labs_first_day l
  on ie.icustay_id = l.icustay_id
left join urine_output_first_day uo
  on ie.icustay_id = uo.icustay_id
left join gcs_first_day gcs
  on ie.icustay_id = gcs.icustay_id
)
, scorecalc as
(
  -- Calculate the final score
  -- note that if the underlying data is missing, the component is null
  -- eventually these are treated as 0 (normal), but knowing when data is missing is useful for debugging
  select icustay_id
  -- Respiration
  , case
      when pao2fio2_vent_min   < 100 then 4
      when pao2fio2_vent_min   < 200 then 3
      when pao2fio2_novent_min < 300 then 2
      when pao2fio2_novent_min < 400 then 1
      when coalesce(pao2fio2_vent_min, pao2fio2_novent_min) is null then null
      else 0
    end as respiration

  -- Coagulation
  , case
      when platelet_min < 20  then 4
      when platelet_min < 50  then 3
      when platelet_min < 100 then 2
      when platelet_min < 150 then 1
      when platelet_min is null then null
      else 0
    end as coagulation

  -- Liver
  , case
      -- Bilirubin checks in mg/dL
        when bilirubin_max >= 12.0 then 4
        when bilirubin_max >= 6.0  then 3
        when bilirubin_max >= 2.0  then 2
        when bilirubin_max >= 1.2  then 1
        when bilirubin_max is null then null
        else 0
      end as liver

  -- Cardiovascular
  , case
      when rate_dopamine > 15 or rate_epinephrine >  0.1 or rate_norepinephrine >  0.1 then 4
      when rate_dopamine >  5 or rate_epinephrine <= 0.1 or rate_norepinephrine <= 0.1 then 3
      when rate_dopamine >  0 or rate_dobutamine > 0 then 2
      when meanbp_min < 70 then 1
      when coalesce(meanbp_min, rate_dopamine, rate_dobutamine, rate_epinephrine, rate_norepinephrine) is null then null
      else 0
    end as cardiovascular

  -- Neurological failure (GCS)
  , case
      when (mingcs >= 13 and mingcs <= 14) then 1
      when (mingcs >= 10 and mingcs <= 12) then 2
      when (mingcs >=  6 and mingcs <=  9) then 3
      when  mingcs <   6 then 4
      when  mingcs is null then null
  else 0 end
    as cns

  -- Renal failure - high creatinine or low urine output
  , case
    when (creatinine_max >= 5.0) then 4
    when  urineoutput < 200 then 4
    when (creatinine_max >= 3.5 and creatinine_max < 5.0) then 3
    when  urineoutput < 500 then 3
    when (creatinine_max >= 2.0 and creatinine_max < 3.5) then 2
    when (creatinine_max >= 1.2 and creatinine_max < 2.0) then 1
    when coalesce(urineoutput, creatinine_max) is null then null
  else 0 end
    as renal
  from scorecomp
)
select ie.subject_id, ie.hadm_id, ie.icustay_id
  -- Combine all the scores to get SOFA
  -- Impute 0 if the score is missing
  , coalesce(respiration,0)
  + coalesce(coagulation,0)
  + coalesce(liver,0)
  + coalesce(cardiovascular,0)
  + coalesce(cns,0)
  + coalesce(renal,0)
  as SOFA
, respiration
, coagulation
, liver
, cardiovascular
, cns
, renal
FROM icustays ie
left join scorecalc s
  on ie.icustay_id = s.icustay_id
order by ie.icustay_id;
"""

data = pd.read_sql_query(query,con)
data

,subject_id,hadm_id,icustay_id,sofa,respiration,coagulation,liver,cardiovascular,cns,renal
0,55973,152234,200001,6,NaN,1.0,0.0,1.0,1.0,3.0
1,27513,163557,200003,6,2.0,1.0,2.0,1.0,0.0,0.0
2,10950,189514,200006,1,NaN,0.0,NaN,1.0,0.0,0.0
3,20707,129310,200007,1,NaN,0.0,NaN,1.0,0.0,0.0
4,29904,129607,200009,3,0.0,2.0,NaN,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
61527,13620,169431,299993,0,NaN,0.0,0.0,0.0,0.0,0.0
61528,10718,177406,299994,5,NaN,0.0,NaN,1.0,0.0,4.0
61529,28775,134959,299995,3,2.0,0.0,0.0,1.0,0.0,0.0
61530,69587,158288,299998,4,3.0,0.0,NaN,1.0,0.0,0.0


In [23]:
sofa = data[['icustay_id','sofa']]
sofa

,icustay_id,sofa
0,200001,6
1,200003,6
2,200006,1
3,200007,1
4,200009,3
...,...,...
61527,299993,0
61528,299994,5
61529,299995,3
61530,299998,4


# Merging for final csv

In [24]:
from functools import partial, reduce

dfs = [months, current, demo, df_weight, df_height, sofa, vitals, labs]
merge = partial(pd.merge, on=['icustay_id'], how='outer')
final = reduce(merge, dfs)
final

,icustay_id,day_28_flag,icu_exp_flag,ethnicity,gender,age,length_of_stay,service_type,weight,height_inches,...,Temperature,BUN,CHLORIDE,CREATININE,HEMOGLOBIN,PLATELET,POTASSIUM,SODIUM,TOTALCO2,WBC
0,243653,0,0,ASIAN,M,0.00,0.0,NICU,NaN,NaN,...,NaN,NaN,NaN,NaN,16.5,302.0,NaN,NaN,NaN,22.0
1,211552,0,0,WHITE,M,76.52,6.0,MICU,96.800003,70.5,...,36.900002,17.0,96.0,1.3,9.7,191.0,3.4,138.0,23.0,10.5
2,294638,0,0,WHITE,F,47.84,1.0,MICU,NaN,NaN,...,36.666667,16.0,106.0,0.4,10.5,258.0,3.8,138.0,21.0,7.3
3,214757,0,0,ASIAN,M,0.00,0.0,NICU,NaN,NaN,...,NaN,NaN,NaN,NaN,14.9,309.0,NaN,NaN,NaN,13.9
4,228232,0,0,WHITE,F,65.94,3.0,SICU,NaN,NaN,...,36.555557,73.0,101.0,5.1,12.4,168.0,4.7,130.0,14.0,4.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61527,279638,0,0,WHITE,M,53.81,11.0,MICU,71.000000,71,...,37.666667,17.0,105.0,1.0,8.2,948.0,4.2,136.0,23.0,20.4
61528,226241,0,0,WHITE,M,47.73,3.0,TSICU,100.500000,NaN,...,37.027778,20.0,109.0,0.9,8.9,351.0,3.7,145.0,30.0,7.7
61529,242052,0,0,WHITE,F,65.77,1.0,MICU,65.400000,NaN,...,36.555556,44.0,105.0,0.8,10.4,132.0,4.3,137.0,26.0,16.0
61530,229633,0,0,WHITE,F,88.70,2.0,CSRU,68.000000,62.5,...,36.944444,11.0,98.0,0.4,8.9,124.0,4.2,137.0,33.0,6.4


In [25]:
final.to_csv('data.csv',index=False)